# About



`ITS_LIVE_TOOL` is a package designed to aid users working with the [Inter-mission Time Series of Land Ice Velocity and Elevation](link) (ITS_LIVE) dataset. The package provides functions for accessing data as well as various methods to process ITS_LIVE observations. This notebook will demonstrate various elements of the package and walk through the steps of a typical workflow using ITS_LIVE_TOOL. 

## Overview 

```{note}
keep this overview here? or move to readme doc
```
The tools in the package fall into two main categories: 1) data access + organization, 2) data processing. 

### Data Access + Organization

#### 1. Interactive map widget

This is an interactive widget design to streamline access ITS_LIVE image pair ice velocity time series. 

#### 2. `Glacier`, `Glacier_Centerline`, `Glacier_Point` objects

These are provided to store and keep track of different types of data related to individual units of analysis such as points, centerlines or full glacier surface areas. 

This roadmap document will first demonstrate data access and organization tools before demonstrating the processing tools. 

### Data Processing

We demonstrate and make available two processing routines. Be sure to check out the accompanying [book]() and consider if either of these are appropriate for your data and use case. Note that these methods are in active development and thus should be considered in *beta* phase. Please perform your own data inspection and due diligence if implementing these methods. 

#### Inversion

**to add**
Description -- link to full description and examples


#### Gaussian Process Regression 

**to add**
Description -- link to full description and examples


### Processing

We demonstrate and make available two processing routines. Be sure to check out the accompanying [book]() and consider if either of these are appropriate for your data and use case. Note that these methods are in active development and thus should be considered in *beta* phase. Please perform your own data inspection and due diligence if implementing these methods. 

In [ ]:
from ITS_LIVE_TOOL import datacube_tools, interactive, obj_setup

In [ ]:
import os
import numpy as np
import pyproj
import matplotlib.path as path
import s3fs
import zarr
import matplotlib.pyplot as plt
import scipy
from datetime import timedelta
from tqdm import tqdm
import xarray as xr
import re
import pandas as pd
import geopandas as gpd
import matplotlib.path as mplp
import ipyleaflet as ipyl
from ipyleaflet import WMSLayer
import ipywidgets as ipyw
import json
import pandas as pd
from ipyleaflet import Map, WMSLayer, basemaps
from ipywidgets import HTML
from owslib.wms import WebMapService

## Install

Install this package using the below command 

```{note}
someday we hope to have a pip or conda install, for now use pip install git+ github repo url
```

# Section 1: Data Access + Organization

### How to use

There are two ways to access data using ITS_LIVE_TOOL. The first way is through the interactive widget. This is great for exploratory analysis. The second way is by specifying an RGI ID and point coordinates manually. This is useful if you already know which glacier(s) you want to examine and simply want to pass a list of RGI IDs and coordinates. We'll first demonstrate using the interactive widget.

### Interactive data selection widget

First, use the interactive map to select data. Do this by right- and left-clicking on the map location where you'd like to access data. 
This will return an object containing the coordinates of the point you clicked, a `geopandas.GeoDataFrame` of the [RGI7](http://www.glims.org/rgi_user_guide/welcome.html) data for that glacier and the URL of the ITS_LIVE granule covering the clicked point. 

```{note}
If the glacier you want to study lies in multiple ITS_LIVE granules, or you'd like to look at multiple glaciers, click in multiple points and the output objects will be appended with each click. 
```

In [ ]:
data_map = interactive.Widget()

In [ ]:
data_map.display()

### Create data objects

Once you have made your selection(s) on the map, use the following methods to create objects that will store your data

#### If you've only selected one glacier, create individual objects as follows: 

In [ ]:
data_map.urls

['http://its-live-data.s3.amazonaws.com/datacubes/v2/N50W140/ITS_LIVE_vel_EPSG3413_G0120_X-3350000_Y350000.zarr',
 'http://its-live-data.s3.amazonaws.com/datacubes/v2/N60W140/ITS_LIVE_vel_EPSG3413_G0120_X-3250000_Y350000.zarr',
 'http://its-live-data.s3.amazonaws.com/datacubes/v2/N50W130/ITS_LIVE_vel_EPSG3413_G0120_X-3350000_Y250000.zarr',
 'http://its-live-data.s3.amazonaws.com/datacubes/v2/N60W130/ITS_LIVE_vel_EPSG3413_G0120_X-3250000_Y250000.zarr']

In [ ]:
glacier = obj_setup.create_glacier_from_click(data_map, 0)

In [ ]:
#glacier.itslive_url

'http://its-live-data.s3.amazonaws.com/datacubes/v2/N50W140/ITS_LIVE_vel_EPSG3413_G0120_X-3350000_Y350000.zarr'

In [ ]:
glacier_point = obj_setup.create_glacier_point_from_click(data_map, 0, 'phony label')

original xy [-143.04795214410368, 60.37791914446092] 4326 maps to datacube (-3247134.1548154885, 459126.5490621218) EPSG:3413
original xy [-143.04795214410368, 60.37791914446092] 4326 maps to datacube (-3247134.1548154885, 459126.5490621218) EPSG:3413
subset and load at  63.19 seconds



KeyboardInterrupt



In [ ]:
glacier_centerline = obj_setup.create_glacier_centerline_from_click(data_map, 0)

As you can see, these objects all store data related to the same glacier that was selected, but at different spatial scales. The `glacier` object contains an RGIID, name and the RGI outline and attributes for that glacier. 

In [ ]:
glacier.outline_prj

,id,CENLON,ZMAX,BGNDATE,ZMIN,RGIID,ASPECT,CENLAT,SLOPE,ZMED,...,TERMTYPE,O2REGION,STATUS,ENDDATE,FORM,SURGING,GLIMSID,O1REGION,NAME,geometry
0,RGI_Alaska.13631,-142.072,5381,20100910,3,RGI60-01.13635,273,60.461,9,1525,...,2,5,0,-9999999,0,9,G217928E60461N,1,Bering Glacier,"MULTIPOLYGON (((506808.975 6723108.546, 506874..."


The `glacier_point` object also has the RGI ID and name inherited from the map widget but additionally contains ITS_LIVE image pair velocity time series data for the selected point as well as a 3x3 pixel cube surrounding the point. 

In [ ]:
glacier_point.cube_around_point

<xarray.Dataset>
Dimensions:                (mid_date: 21894, x: 3, y: 4)
Coordinates:
  * mid_date               (mid_date) datetime64[ns] 2020-10-27T20:55:56.2010...
  * x                      (x) float64 -3.247e+06 -3.247e+06 -3.247e+06
  * y                      (y) float64 4.593e+05 4.592e+05 4.591e+05 4.589e+05
    step_count             (mid_date) int64 0 5 12 14 ... 76928 76941 76952
    mapping                int64 0
Data variables:
    v                      (mid_date, y, x) float32 nan 182.0 nan ... 11.0 nan
    vy                     (mid_date, y, x) float32 nan -131.0 nan ... 11.0 nan
    vx                     (mid_date, y, x) float32 nan -126.0 nan ... 2.0 nan
    v_error                (mid_date, y, x) float32 nan 53.0 nan ... nan 6.0 nan
    satellite_img1         (mid_date) object '2A' '2A' '2A' '2B' ... '7' '7' '7'
    satellite_img2         (mid_date) object '2A' '2A' '2B' '2B' ... '8' '8' '8'
    acquisition_date_img1  (mid_date) datetime64[ns] 2020-10-17T20:55:01 ... ...
    acquisition_date_img2  (mid_date) datetime64[ns] 2020-11-06T20:56:51.0000...
    img_separation         (mid_date) float64 21.0 500.0 535.0 ... 217.0 312.0
Attributes: (12/19)
    Conventions:                CF-1.8
    GDAL_AREA_OR_POINT:         Area
    author:                     ITS_LIVE, a NASA MEaSUREs project (its-live.j...
    autoRIFT_parameter_file:    http://its-live-data.s3.amazonaws.com/autorif...
    datacube_software_version:  1.0
    date_created:               25-Sep-2023 22:55:04
    ...                         ...
    s3:                         s3://its-live-data/datacubes/v2/N60W140/ITS_L...
    skipped_granules:           s3://its-live-data/datacubes/v2/N60W140/ITS_L...
    time_standard_img1:         UTC
    time_standard_img2:         UTC
    title:                      ITS_LIVE datacube of image pair velocities
    url:                        https://its-live-data.s3.amazonaws.com/datacu...

The `glacier_centerline` object contains the same RGIID and name information as the first two objects. In addition, it contains OGGM centerline data that are stored as geopandas GeodataFrames.

In [ ]:
glacier_centerline.main_centerline

,RGIID,SEGMENT_ID,LE_SEGMENT,MAIN,geometry
19617,RGI60-01.13635,40,212562.0,1,"LINESTRING (522733.337 6719455.665, 522730.077..."


Each of these objects are meant to act as cotainers to intuitively and efficiently store different types of data as you process and continue to work with ITS_LIVE data. 

The previous steps demonstrated the data access and organization functionality of ITS_LIVE_TOOl. The subsequent sections of this notebook will demonstrate different processing methodologies for working with this dataset. 

#### If you've selected multiple glaciers, follow these examples:

In [ ]:
glacier0, glacier1 = obj_setup.create_multiple_glacier_objs(data_map)[0], obj_setup.create_multiple_glacier_objs(data_map)[1]

In [ ]:
glacier_point_ls = obj_setup.create_multiple_glacier_point_objs(data_map)
glacier_pt0, glacier_pt1 = glacier_point_ls[0], glacier_point_ls[1]

original xy [-141.34499544222683, 60.45529919026818] 4326 maps to datacube (-3250454.7365773427, 361437.5235576865) EPSG:3413
original xy [-141.34499544222683, 60.45529919026818] 4326 maps to datacube (-3250454.7365773427, 361437.5235576865) EPSG:3413
subset and load at  58.26 seconds
original xy [-141.32575672759108, 60.65848956101137] 4326 maps to datacube (-3227246.8706271336, 357759.90272181097) EPSG:3413
original xy [-141.32575672759108, 60.65848956101137] 4326 maps to datacube (-3227246.8706271336, 357759.90272181097) EPSG:3413
subset and load at  60.11 seconds


### Manually create `Glacier`, `Glacier_Point`, `Glacier_Centerline` objects

Create objects that contain the clicked data from the above map widget.  
These are `coords`, `gpdf`, `urls`  


In [ ]:
coords, gpdf, urls = obj_setup.return_clicked_info(data_map)

You have 2 glaciers selected


In [ ]:
coords

[[60.37791914446092, -143.04795214410368],
 [60.36705675717361, -143.64435229781114]]

In [ ]:
label = 'label'
var_ls = ['v','vy','vx','v_error','mapping','satellite_img1','satellite_img2','acquisition_date_img1', 'acquisition_date_img2']

In [ ]:
glacier_manual = obj_setup.Glacier(gpdf[0]['NAME'].iloc[0], gpdf[0]['RGIID'].iloc[0], str(gpdf[0].estimate_utm_crs()), 'manual', rgi_outline_from_widget = None)

In [ ]:
glacier_manual.name

'Bering Glacier'

In [ ]:
glacier_manual.rgi_id

'RGI60-01.13635'

In [ ]:
glacier_manual.outline_prj.crs

<Projected CRS: EPSG:32607>
Name: WGS 84 / UTM zone 7N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 144°W and 138°W, northern hemisphere between equator and 84°N, onshore and offshore. Canada - British Columbia (BC); Yukon. United States (USA) - Alaska (AK).
- bounds: (-144.0, 0.0, -138.0, 84.0)
Coordinate Operation:
- name: UTM zone 7N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
glacier_pt = obj_setup.Glacier_Point(glacier.name, 'acc. zone', glacier.rgi_id, [coords[0][1], coords[0][0]], var_ls)

original xy [-143.04795214410368, 60.37791914446092] 4326 maps to datacube (-3247134.1548154885, 459126.5490621218) EPSG:3413
original xy [-143.04795214410368, 60.37791914446092] 4326 maps to datacube (-3247134.1548154885, 459126.5490621218) EPSG:3413
subset and load at  62.53 seconds


In [ ]:
glacier_centerline = obj_setup.Glacier_Centerline('Jorge Montt', 'RGI60-17.06074')

### `Glacier_Point`

This object inherits name and RGIID attributes from the `Glacier` object described aboev (or you can input them manually). It takes the coordinate information specified in the widget and list of variables of interest and returns an object containing ITS_LIVE image pair time series datasets for the specified point anda 3x3 pixel cube surrounding the specified point. Check out the docs for a description of all of the attributes and methods associated with this object. 

In [ ]:
coords

[[61.445551290803365, -147.96202722979135]]

In [ ]:
glacier_pt = obj_setup.Glacier_Point(glacier.name, 'acc. zone', glacier.rgi_id, [coords[0][1], coords[0][0]], var_ls)

original xy [-147.96202722979135, 61.445551290803365] 4326 maps to datacube (-3075868.4013177734, 707973.3167825146) EPSG:3413
original xy [-147.96202722979135, 61.445551290803365] 4326 maps to datacube (-3075868.4013177734, 707973.3167825146) EPSG:3413
subset and load at  48.89 seconds


In [ ]:
glacier_pt.cube_around_point

<xarray.Dataset>
Dimensions:                (mid_date: 13831, x: 3, y: 3)
Coordinates:
  * mid_date               (mid_date) datetime64[ns] 2020-02-26T21:15:44.1906...
  * x                      (x) float64 -3.076e+06 -3.076e+06 -3.076e+06
  * y                      (y) float64 7.081e+05 7.079e+05 7.078e+05
    step_count             (mid_date) int64 1 6 7 8 ... 53266 53268 53273 53285
    mapping                int64 0
Data variables:
    v                      (mid_date, y, x) float32 nan nan nan ... nan nan nan
    vy                     (mid_date, y, x) float32 nan nan nan ... nan nan nan
    vx                     (mid_date, y, x) float32 nan nan nan ... nan nan nan
    v_error                (mid_date, y, x) float32 nan nan nan ... nan nan nan
    satellite_img1         (mid_date) object '2B' '5' '2B' '7' ... '7' '8' '8'
    satellite_img2         (mid_date) object '2B' '5' '2B' '7' ... '8' '8' '8'
    acquisition_date_img1  (mid_date) datetime64[ns] 2019-06-26T21:15:19.0000...
    acquisition_date_img2  (mid_date) datetime64[ns] 2020-10-28T21:16:09 ... ...
    img_separation         (mid_date) float64 491.0 305.0 340.0 ... 352.0 17.0
Attributes: (12/19)
    Conventions:                CF-1.8
    GDAL_AREA_OR_POINT:         Area
    author:                     ITS_LIVE, a NASA MEaSUREs project (its-live.j...
    autoRIFT_parameter_file:    http://its-live-data.s3.amazonaws.com/autorif...
    datacube_software_version:  1.0
    date_created:               03-Oct-2023 04:28:15
    ...                         ...
    s3:                         s3://its-live-data/datacubes/v2/N60W140/ITS_L...
    skipped_granules:           s3://its-live-data/datacubes/v2/N60W140/ITS_L...
    time_standard_img1:         UTC
    time_standard_img2:         UTC
    title:                      ITS_LIVE datacube of image pair velocities
    url:                        https://its-live-data.s3.amazonaws.com/datacu...

In [ ]:
glacier_pt

<xarray.Dataset>
Dimensions:                (mid_date: 3906, y: 83, x: 83)
Coordinates:
    mapping                int64 0
  * mid_date               (mid_date) datetime64[ns] 2020-01-26T14:27:11.2001...
  * x                      (x) float64 6.017e+05 6.019e+05 ... 6.116e+05
  * y                      (y) float64 4.633e+06 4.633e+06 ... 4.623e+06
    step_count             (mid_date) int64 7 29 30 38 ... 31281 31283 31302
Data variables:
    v                      (mid_date, y, x) float32 37.0 25.0 9.0 ... nan nan
    vy                     (mid_date, y, x) float32 -2.0 -11.0 2.0 ... nan nan
    vx                     (mid_date, y, x) float32 36.0 23.0 9.0 ... nan nan
    v_error                (mid_date, y, x) float32 26.0 26.0 26.0 ... nan nan
    satellite_img1         (mid_date) object '2A' '2A' '1A' '2A' ... '8' '8' '8'
    satellite_img2         (mid_date) object '2A' '2A' '1A' '2B' ... '7' '9' '8'
    acquisition_date_img1  (mid_date) datetime64[ns] 2020-01-01T14:27:30.9999...
    acquisition_date_img2  (mid_date) datetime64[ns] 2020-02-20T14:26:51 ... ...
    img_separation         (mid_date) float64 50.0 260.0 12.0 ... 536.0 16.0
Attributes: (12/19)
    Conventions:                CF-1.8
    GDAL_AREA_OR_POINT:         Area
    author:                     ITS_LIVE, a NASA MEaSUREs project (its-live.j...
    autoRIFT_parameter_file:    http://its-live-data.s3.amazonaws.com/autorif...
    datacube_software_version:  1.0
    date_created:               25-Sep-2023 21:36:13
    ...                         ...
    s3:                         s3://its-live-data/datacubes/v2/S40W070/ITS_L...
    skipped_granules:           s3://its-live-data/datacubes/v2/S40W070/ITS_L...
    time_standard_img1:         UTC
    time_standard_img2:         UTC
    title:                      ITS_LIVE datacube of image pair velocities
    url:                        https://its-live-data.s3.amazonaws.com/datacu...

### `Glacier_Centerline`

Similarly to the `Glacier_Point` object, a `Glacier_Centerline` object inherits some basic information from a `Glacier` object, but this can also be specified manually if you don't want to create a `Glacier` method beforehand. A `Glacier_Centerline` object has attributes called `centerlines` and `main_centerline`. These are both `Geopandas.GeoDataframe` objects containing OGGM flowline data associated with this glacier. They also contain ITS_LIVE image pair velocity time series clipped to the centerline (**still being implemented**).

In [ ]:
glacier_centerline = obj_setup.Glacier_Centerline('Jorge Montt', 'RGI60-17.06074')

In [ ]:
glacier_centerline

<ITS_LIVE_TOOL.obj_setup.Glacier_Centerline>

Each of these objects are meant to act as cotainers to intuitively and efficiently store different types of data as you process and continue to work with ITS_LIVE data. 

The previous steps demonstrated the data access and organization functionality of ITS_LIVE_TOOl. The subsequent sections of this notebook will demonstrate different processing methodologies for working with this dataset. 

# Section 2: Data Processing

## Gaussian Process Regression

We demonstrate a method to develop and apply a Gaussian Process model on ITS_LIVE point time series data. Gaussian Process are a non-parametric, Bayesion regression method to model a latent function such as glacier surface velocity with respect to time, given observations of ice movement over different time periods. 

```{note}
This method is in active development and should currently be considered to be in 'beta' mode. We provide an example of its implementation but stress that it may not be suitable to apply in other scenarios. Please carefully inspect your dataset and the model parameters to be sure it is appropriate before using in your workflows
```

### Problem:

ITS_LIVE image pair ice velocity time series provide estimates of ice movement over a broad range of temporal baselines (number of days between image pairs). The estimates describe different types of ice movement given different temporal baselines and contain noise inherent to sensor algorithmic limitations. Given this complex, noisy dataset, we would like to employ a regression approach to reach an estimate of an underlying velocity function with inherent uncertainty quantifications.

### Raw data

![ITS_LIVE velocity observations for a single point](https://github.com/e-marshall/ITS_LIVE_TOOL/blob/main/figs/baltoro_raw_data_example.png?raw=true)

### Gaussian Process model predictions

![Velocity time series predicted by Gaussian Process model based on ITS_LIVE velocity observations](https://github.com/e-marshall/ITS_LIVE_TOOL/blob/main/figs/baltoro_gp_example_results.png?raw=true)

```{note}
Put inversion and other processing steps here? 
```

# Datacube Inversion
# Theory of velocity interpolation for a single point time series
<br>
<br>

#### **Problem:**

*Find a smooth regularly spaced timeseries of velocities given a set of average measurements of possibly overlapping and variable length time intervals*

<br>
<br>

#### **Start: we formulate it as an inverse problem**

*Find a velocity vector $\mathbf{v}$ that minimizes the following functional:*

<br>

$U(\mathbf{v},\lambda) = \|\mathbf{vInv}\|_r + \lambda \|A_{m}\mathbf{vInv} - \mathbf{vObs}\|$

<br>

- $\mathbf{vObs}$: velocity measurements

- $\mathbf{vInv}$: vector of speeds that we are solving for at a regular interval (for example daily)

- $\|.\|_r$: is a roughness norm or a measure of the first derivative

- $\lambda$: can be seen as a Lagrange multiplier (it enforces the condition in the expression behind it), or one can think of it as a relative weight of how well we want it to fit the data versus how smooth the resulting interpolated velocity should be.

- $\mathbf{A_{m}}$: indexing matrix that specifies over which time interval a velocity was measured (and averages it appropriately).

<br>

#### **Design Matrix part 1: Date Span Matrix**

For a simple example, assume you want daily velocities for 6 days, and you have a measurement of the average velocity from day 1 to day 4 ($d_1$) and from day 3 to day 5 ($d_2$). We then have:

$$A_{m} = \begin{bmatrix}
		1/4 & 1/4 & 1/4 & 1/4 & 0 & 0 \\ 
		0 & 0 & 1/3 & 1/3 & 1/3 & 0
		\end{bmatrix}$$

<br>

**Dimensions:** The number of rows is equal to the number of measurements $N$ (same dimension as $\mathbf{vObs}$) and the number of columns $M$ (same dimension as $\mathbf{vInv}$).

<br>

#### **Design Matrix part 2: Regularization Matrix**

In order to regularize the matrix in time, we create a matrix of regularization $\mathbf{reg_{mat}}$ which dimensions are $(M-D)$ x $M$ (D is the degree of the derivative used for the regularization), for which each entry will be depending on the $\lambda$ term and the order of the term we apply the regularization to (approximation of finite differences).

**1st derivative regularization:** $f'(x) \approx \frac{f_{i+1} - f_{i}}{h}$  Matrix size: $(M-1)$ x $M$ 

Which would result in: $$reg_{mat} = \begin{bmatrix}
										-\frac{\lambda}{h} & \frac{\lambda}{h} & 0 & 0 & 0 & 0 \\ 
										0 & -\frac{\lambda}{h} & \frac{\lambda}{h}& 0 & 0 & 0 \\
										0 & 0 & \frac{\lambda}{h} & \frac{\lambda}{h}& 0 & 0 \\
										0 & 0 & 0 & -\frac{\lambda}{h} & \frac{\lambda}{h}& 0 \\
										0 & 0 & 0 & 0 & -\frac{\lambda}{h} & \frac{\lambda}{h} \\
										\end{bmatrix}$$

<br>

**2nd derivative regularization:** $f''(x) \approx \frac{f_{i+2} - 2f_{i+1} + f_{i}}{h}$  Matrix size: $(M-2)$ x $M$ 

Which would result in: $$reg_{mat} = \begin{bmatrix}
										\frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 & 0 \\ 
										0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 \\
										0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 \\
										0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} \\
										\end{bmatrix}$$

<br>

#### **Design Matrix part 3: Assemble the Design Matrix**
And we finally obtain our main design matrix (with 2nd derivative), which is a concatenation of $A_{m}$ and $reg_{mat}$:

$$A_{reg} = \begin{bmatrix}				\\ & & A_{m} & & & \\ \\
										\frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 & 0 \\ 
										0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 \\
										0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 \\
										0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} \\
										\end{bmatrix}$$

<br>

#### **Extend the vector of observations**

Because we added a regularization matrix to our design matrix, we need to extend our vector of observations $\mathbf{vObs}$ so we can calculate the inversion:

$$\begin{bmatrix} v_{0} \\ v_{1} \\ ... \\ v_{n} \end{bmatrix} to \begin{bmatrix} v_{0} \\ v_{1} \\ ... \\ v_{n} \\ 0 \\ 0 \\ ... \\ 0 \end{bmatrix} $$



With the amount of 0s appended being $(M-1)$

The obtained velocity vector is then the size of the regular matrix in time. 
(We can add a spatial regularization but it increases the computation time by 3.)

<br>

#### **Final Part: Solve the Linear System of Equations**

$A_{reg}^{T}A_{reg}\mathbf{vInv}= A_{reg}^{T}\mathbf{vObs}$

<br>

Using a linear solver (`np.linalg.solve()`, `scipy.linalg.solve()`, `torch.linalg.solve()`) find $\mathbf{vInv}$ satisfying the equation, which is equivalent to solving: $U(\mathbf{v},\lambda) = \|\mathbf{vInv}\|_r + \lambda \|A_{m}\mathbf{vInv} - \mathbf{vObs}\|$. 

We can use a linear solver because the matrix $A_{reg}^{T}A_{reg}$ is square, symmetrical positive.

<br>
<br>

# Inversion applied to ITS_LIVE

#### **ITS_LIVE primary issue:**

ITS_LIVE is a great dataset that achieves high-density measurements globally, on a long timescale, using the same sensors and method.
However it suffers from one main problem: **mid_dates**, its attribution of a timestamp for a measurement.

<br>

**Example 1:** ITS_LIVE uses two images to calculate the velocity of the ice. 1st image is acquired on *January 1st*, 2nd image is acquired on *January 15th*.

We obtain an average velocity over 15 days: $v = \frac{displacement}{15 days} = 10 m/day$

The problem is that ITS_LIVE will attribute this velocity to a **precise point in time** rather than a **period of time**: the velocity of $10 m/day$ is attributed to *January 7th* at Noon. This is physically wrong.

It is difficult to work with periods of time, while precise points in time are much more convenient, which is probably why ITS_LIVE organizes its data like that.

<br>

**Example 2:** We now have 2 measurements of velocities:
- **V1**: $10 m/day$, timestamp: *January 7th*, Date 1st image acquisition: *January 1st*, Date 2nd image acquisition: *January 15th*  
- **V2**: $5 m/day$, timestamp: *January 12th*, Date 1st image acquisition: *January 5th*, Date 2nd image acquisition: *January 19th*  

ITS_LIVE would put only 2 timestamps. However there is more information than just during those two days. The intersection between the two measurements procures information on the average velocity between *January 5th* and *January 15th*. Because we now have a fragmentation of each acquisition into smaller segments, we can also derive refined average velocities for the following periods of time:
- *January 1st* and *January 5th*
- *January 5th* and *January 7th*
- *January 7th* and *January 12th*
- *January 12th* and *January 15th*
- *January 15th* and *January 19th*

<br>

#### **Inversion theory applied to ITS_LIVE:**

Let's consider the same setup at in **Example 2**.
We want to linearize in time to force a sampling of velocities 2 days. Thus, our matrix $A_{m}$ will have 2 rows (we have 2 average velocities given by ITS_LIVE), and 10 columns (19 days between the first and last acquisition dates: *January 1st* and *January 19th*, divided by 2 because we force 2 days sampling):

$$dates = \begin{bmatrix}
		1st & 3rd & 5th & 7th & 9th & 11th & 13th & 15th & 17th & 19th
		\end{bmatrix}$$

$$A_{m} = \begin{bmatrix}
		1/15 & 1/15 & 1/15 & 1/15 & 1/15 & 1/15 & 1/15 & 1/15 & 0    &    0   \\ 
		0    &    0 & 1/14 & 1/14 & 1/14 & 1/14 & 1/14 & 1/14 & 1/14 & 1/14
		\end{bmatrix}$$
<br>

In the matrix above, each column is paired with a date (matrix 'dates). The first row of $A_{m}$ corresponds to the first velocity measurement. Because this measurement is the average velocity from *January 1st* to *January 15th*, every day in between has *1/15* attributed to it in the matrix (15 days, each contributing equally to the velocity observed). 
The 2nd row, representing the 2nd velocity measurement (or observed) spans 14 days hence why the values there are *1/14*. Note that when a day is not covered by the velocity time period, the value is 0.

<br>

**Assemble the design matrix:** (regularization in time with 2nd derivative, Jacobian):


$$A_{reg} = \begin{bmatrix}				1/15 & 1/15 & 1/15 & 1/15 & 1/15 & 1/15 & 1/15 & 1/15 & 0    &    0  \\ 
										0    &    0 & 1/14 & 1/14 & 1/14 & 1/14 & 1/14 & 1/14 & 1/14 & 1/14	 \\
										\frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ 
										0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 & 0 & 0 & 0 & 0 \\
										0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 & 0 & 0 & 0 \\
										0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 & 0 & 0 \\
										0 & 0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 & 0 \\
										0 & 0 & 0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 \\
										0 & 0 & 0 & 0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 \\
										0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}}\\
										\end{bmatrix}$$

<br>

**Extend the vector $\mathbf{vObs}$:**

$$\mathbf{vObs} = \begin{bmatrix} v1 \\ v2 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \end{bmatrix} $$

**Solve the linear system of equation:**

$$\begin{bmatrix}				        1/15 & 1/15 & 1/15 & 1/15 & 1/15 & 1/15 & 1/15 & 1/15 & 0    &    0  \\ 
										0    &    0 & 1/14 & 1/14 & 1/14 & 1/14 & 1/14 & 1/14 & 1/14 & 1/14	 \\
										\frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ 
										0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 & 0 & 0 & 0 & 0 \\
										0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 & 0 & 0 & 0 \\
										0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 & 0 & 0 \\
										0 & 0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 & 0 \\
										0 & 0 & 0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 \\
										0 & 0 & 0 & 0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 \\
										0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}}\\
										\end{bmatrix} 
										
										\begin{bmatrix}				
										1/15 & 0 & \frac{\lambda}{h^{2}} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
										1/15 & 0 & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 & 0 & 0 & 0 & 0\\
										1/15 & 1/14 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 & 0 & 0 \\
										1/15 & 1/14 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 & 0 \\
										1/15 & 1/14 & 0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 \\
										1/15 & 1/14 & 0 & 0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 \\
										1/15 & 1/14 & 0 & 0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}}  & 0\\
										1/15 & 1/14 & 0 & 0 & 0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}}\\
										0    & 1/14 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{\lambda}{h^{2}} & 2\frac{\lambda}{h^{2}}\\
										0    & 1/14 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{\lambda}{h^{2}}\\ 

										\end{bmatrix}\mathbf{vInv}$$
										
$$ = $$
$$								\begin{bmatrix}				
										1/15 & 0 & \frac{\lambda}{h^{2}} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
										1/15 & 0 & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 & 0 & 0 & 0 & 0\\
										1/15 & 1/14 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 & 0 & 0 \\
										1/15 & 1/14 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 & 0 \\
										1/15 & 1/14 & 0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 & 0 \\
										1/15 & 1/14 & 0 & 0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}} & 0 \\
										1/15 & 1/14 & 0 & 0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}}  & 0\\
										1/15 & 1/14 & 0 & 0 & 0 & 0 & 0 & \frac{\lambda}{h^{2}} & -2\frac{\lambda}{h^{2}} & \frac{\lambda}{h^{2}}\\
										0    & 1/14 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{\lambda}{h^{2}} & 2\frac{\lambda}{h^{2}}\\
										0    & 1/14 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{\lambda}{h^{2}}\\ 

										\end{bmatrix}\begin{bmatrix} v1 \\ v2 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \end{bmatrix} 
										$$

**You end up with $\mathbf{vInv}$**









<br>

# Input your parameters below

In [ ]:
mission = None # 'None' if you want all the data, 'S1' for Sentinel-1 only, 'L' for Landsat only, etc.. .
lamb = 10 # Smoothing coefficient: the higher the value, the more the inversion favors a smooth output. BAD for surging glaciers, GOOD for non-surging glaciers.
derivative = 2 # Derivative degree for the inversion. Doesn't change much unless you have a specific reasong to choose 1 or 2 (1st or 2nd derivative, Hessian of Jacobian)
day_interval = 12 # Amount of days in between each inversion value. The higher, the faster the inversion. But you also lose in temporal resolution. 12 here because Sentinel-1 repeat-time is 12.
sdate = None # Start date, format 'YYYY-MM-DD' OR None if you want to grab the entire timeseries available
edate = None # End date, format 'YYYY-MM-DD' OR None if you want to grab the entire timeseries available
GPU = True # None if you want to run it on CPU

## Select your data

#### **Instructions:**

- Right-click on a glacier to get its RGI ID. It should illuminate in blue once you have selected it. You can select multiple glaciers.

    **WARNING:** if you are on a web browser (Cryocloud, OpenScienceLab, etc...): right-click will open a pop-up menu. Simply left-click somewhere on your screen to make it disappear.

- Left-click on the datacube (*grey* overlay, *red* when you hover your mouse over it) you want to download. You can select multiple datacubes.
    When a datacube URL has been fetched, it should print it below the GUI.





In [ ]:
urls = []

In [ ]:
data_map = interactive.Widget()

In [ ]:
data_map.display()

In [ ]:
try: 
    coords, gpdf, urls = interactive.return_clicked_info(data_map)
except:
    pass

1


In [ ]:
##### EMMA IN NOTEBOOK 2 ADD 'GeoData' to the import line: from ipyleaflet import Map, WMSLayer, basemaps, GeoData

t = []
wms_url = "https://glims.org/geoserver/ows?SERVICE=WMS&"
wms_layer = WMSLayer(
    url=wms_url,
    layers='GLIMS:RGI',
    transparent=True,
    format='image/png'
)


# Create a Pandas DataFrame with X and Y coordinates
#data = pd.DataFrame({'X': list(fx_map[::50]), 'Y': list(fy_map[::50])})

# Map and label widgets
map = ipyl.Map(basemap=basemaps.Esri.WorldImagery, center=(0, 0), zoom=2)
label = ipyw.Label(layout=ipyw.Layout(width="100%"))

# Create a list to store clicked URLs
urls = []

# geojson layer with hover handler
with open("catalog_v02.json") as f:
    geojson_data = json.load(f)

for feature in geojson_data["features"]:
    feature["properties"]["style"] = {
        "color": "grey",
        "weight": 1,
        "fillColor": "grey",
        "fillOpacity": 0.5,
    }

geojson_layer = ipyl.GeoJSON(data=geojson_data, hover_style={"fillColor": "red"}, selected_style = {"fillColor": "orange"})

def hover_handler(event=None, feature=None, id=None, properties=None):
    label.value = properties["zarr_url"]

def json_handler(event=None, feature=None, id=None, properties=None):    
    zarr_url = properties.get("zarr_url", "N/A")
    urls.append(zarr_url)
    print(f"Clicked URL: {zarr_url}")
    print("All Clicked URLs:", urls)
    '''
    # Plot in orange the polygon clicked on
    json_shape = [{'type': properties['geometry_epsg']['type'], 'coordinates': properties['geometry_epsg']['coordinates']}]
    geom = [shape(i) for i in json_shape]
    json_shape = gpd.GeoDataFrame({'geometry':geom})  
    json_shape = GeoData(geo_dataframe = json_shape,
                style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                hover_style={'fillColor': 'blue' , 'fillOpacity': 0.2},
                name = 'Cube')

    map.add_layer(json_shape)
    '''

    
    
wms = WebMapService(wms_url)

# Initialize a list to store the geopandas dataframes
gdf_list = []

# Create a function to handle click events
def click_handler(properties=None, **kwargs):


    if kwargs.get('type') == 'contextmenu':
        latlon = kwargs.get('coordinates')
        lat, lon = latlon[0], latlon[1]
        print(f"Clicked at (Lat: {lat}, Lon: {lon})")
        
        # Arrange the coordinates
        
        response = wms.getfeatureinfo(
            layers=['GLIMS:RGI'],
            srs='EPSG:4326',
            bbox=(lon-0.001,lat-0.001,lon+0.001,lat+0.001),
            size=(1,1),
            format='image/jpeg',
            query_layers=['GLIMS:RGI'],
            info_format="application/json",
            xy=(0,0))
        df = gpd.read_file(response)
        gdf_list.append(df)  
        try:
            print(f"You have selected the glacier {df['NAME'].values[0]}, ID: {df['id'].values[0]} ")
        except:
            print(f"This glacier is not recognized by the RGI (maybe an ice-shelf ?) -> Choose another one")
        geo_data = GeoData(geo_dataframe = df,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'blue' , 'fillOpacity': 0.2},
                   name = 'Glacier')

        gdf_list.append(df) 
        map.add_layer(geo_data)

        


geojson_layer.on_hover(hover_handler)
geojson_layer.on_click(json_handler)


map.add(geojson_layer)

# Add points from the DataFrame to the map
#for index, row in data.iterrows():
#    marker = ipyl.Marker(locatfairbanks ash situationion=(row['Y'], row['X']))
#    map.add_layer(marker)

# Enable zoom when scrolling with the mouse
map.scroll_wheel_zoom = True
ipyw.VBox([map, label])


# Add the WMS layer to the map
map.add_layer(wms_layer)

# Add the click event handler to the map
map.on_interaction(click_handler)

map

The following cell ensures you don't have duplicate glaciers in your selection. If so, it will automatically remove them.
It also verifies that you have selected a datacube to download. If not, please re-select it and ensure you are getting a URL after a left-click.

In [ ]:
### EMMA THIS IS THE CELL TO MODIFY

# Get indices of unique geopandas dataframes in case you selected the same glacier twice
unique_values, unique_indices = np.unique(np.array([i['id'] for i in gdf_list]), return_index=True)

# Iterate through the unique indices to recreate the list of geopandas dataframes unique only
gdf_list = [gdf_list[i] for i in unique_indices]

print(f"You have {len(gdf_list)} glaciers selected")

if len(urls) == 0:
    print('Select a datacube to fetch the data !!')

# Print the first dataframe to show the structure of it
gdf_list[0]

Just a peek at what the Randolph Glacier Inventory (RGI) gives us for a glacier object:

In [ ]:
gpdf[0][0]

In [ ]:
urls

['http://its-live-data.s3.amazonaws.com/datacubes/v2/N20E080/ITS_LIVE_vel_EPSG32644_G0120_X750000_Y3250000.zarr']

In [ ]:
try: 
    gpdf_ls = gpdf[0]
except:
    pass

In [ ]:
def make_input_dict(coords, gpdf, urls):
    
    mod_urls = [re.sub(r'http', 's3', url) for url in urls]
    mod_urls = [re.sub(r'\.s3\.amazonaws\.com', '', url) for url in mod_urls]
    
    d = {'coords': coords,
     'gpdf': gpdf,
     'urls': mod_urls
        }
    return d

In [ ]:
try:
    d = make_input_dict(coords, gpdf, URLs)
    urls = d['urls']
except:
    print(' no input data specified, have you made a map selection?')

 no input data specified, have you made a map selection?


The following function creates a dictionnary for a datacube URL. It prepares the many variables we will use during the cube inversion.

In [ ]:
def create_data_dict(urls):
    # Modify the urls so they can be opened by zarr (replace 'http' by 's3' and delete '.s3.amazonaws.com')
    mod_urls = [re.sub(r'http', 's3', url) for url in urls]
    mod_urls = [re.sub(r'\.s3\.amazonaws\.com', '', url) for url in urls]
    
    # Create storing arrays for the coordinates on-glacier
    X_valid = []
    Y_valid = []
    X_tot = []
    Y_tot = []
    
    # Create an empty directoryimport pickle to hold many variables all tied to the datacubes
    data_dict = {}
    

    # We iterate through the different datacubes so they can each have one instance of the variables below
    for mod_urls in urls:
        zarr_store = None # To store the datacube's information and access its variables
        dates = None # To store the dates at which the inversion will give values
        A_m = None # 1st part of the design matrix
        reg_mat_Inv = None # Regularization in time, 2nd part of the design matrix
        mission = None # If you want to invert specifically for one mission in particular ('S1','L8','L9', etc...)
        index_sort = None # Indices representing the sorted dates (from older to most recent)
        inds_mission = None # Indices representing the sorted dates per mission chosen
        ind_tot = None # Indices representing the indices of the pixels on the GOI
        valid_idx = None # Easting and Northing values of the indices above
        proj_cube = None # Projection of the datacube
        mask_dates = None # Mask that filters out dates outside of desired date range
        
        # Create a dictionary entry for the URL with the desired subsets
        data_dict[mod_urls] = {
            'zarr_store': zarr_store,
            'dates_noinv': dates,
            'A_m': A_m,
            'reg_mat_Inv': reg_mat_Inv,
            'mission': mission,
            'index_sort': index_sort,
            'inds_mission': inds_mission,
            'dates': dates,
            'ind_tot': ind_tot,
            'valid_idx': valid_idx,
            'proj_cube': proj_cube,
            'mask_dates': mask_dates
        }
        
    return data_dict, X_valid, Y_valid, X_tot, Y_tot

This function inputs a datacube. It fetches its spatial footprint, and flags which cells that fall on the glacier shape you selected earlier.
The purpose is to create an array that gathers the spatial intersection between all the datacubes and glaciers you selected. 
It also creates a mask so we only download datacube cells on a glacier, and we don't compute the inversion on non-glacier cells in case some get through the filter.

In [ ]:
def get_extents(url, gpdf, X_tot, Y_tot, X_valid, Y_valid, data_dict):# mission, lamb, derivative, day_interval):
    
    #url = input_data_dict['urls'].iloc[0]
    
    # Open the zarr files
    fs = s3fs.S3FileSystem(anon=True)
    store = zarr.open(s3fs.S3Map(url, s3=fs))
   
    # Update the dictionnary
    data_dict[url]['zarr_store'] = store

    # Get the cube's projection
    proj_cube = store.attrs['projection']

    # Load X and Y of the dataset
    X = store['x'][:]
    Y = store['y'][:]

    # Store the arrays in the total list
    X_tot.append(X)
    Y_tot.append(Y)

    # Load dimensions
    shape_arr = store['v'].shape
    
    Xs, Ys = np.meshgrid(X, Y)
    points = np.array((Xs.flatten(), Ys.flatten())).T

    idx_valid = []
    
    for b in range(len(gpdf)):
        mpath = mplp.Path(list(gpdf[b]['geometry'].to_crs(np.int(proj_cube)).boundary.explode(index_parts = True).iloc[0].coords))
        glacier_mask = mpath.contains_points(points).reshape(Xs.shape)
        # Grab the indices of the points inside the glacier
        idx_valid.append(np.array(np.where(glacier_mask==True)))
        
    idx_valid = np.hstack(idx_valid)
    # Store the valid indices
    data_dict[url]['valid_idx'] = idx_valid
    
    # Store the cube projection
    data_dict[url]['proj_cube'] = proj_cube
    
    # Store the coordinates of the valid Xs and Ys
    X_valid.append([Xs[idx_valid[0][i], idx_valid[1][i]] for i in range(len(idx_valid[0]))])
    Y_valid.append([Ys[idx_valid[0][i], idx_valid[1][i]] for i in range(len(idx_valid[0]))])
    
    return X_tot, Y_tot, X_valid, Y_valid

For each datacube you selected, locate the on-glacier cells.
Once it has been done, create an array gathering all the glacier cells you want to invert.


In [ ]:
try:
    data_dict, X_valid, Y_valid, X_tot, Y_tot = create_data_dict(urls)

    for url in tqdm(range(len(urls))):
        X_tot, Y_tot, X_valid, Y_valid = get_extents(urls[url], gpdf_ls, X_tot, Y_tot, X_valid, Y_valid, data_dict)

    # Create Eastings and Northings arrays based on the Eastings and Northings of the datacubes
    X_arr = np.unique(np.hstack(X_tot))
    Y_arr = np.unique(np.hstack(Y_tot))
    
    # Crop to the GOI (so we avoid over-filling our matrix with NaNs)
    x_min = np.where(np.min(np.hstack(X_valid)) == X_arr)[0][0]
    x_max = np.where(np.max(np.hstack(X_valid)) == X_arr)[0][0]
    y_min = np.where(np.min(np.hstack(Y_valid)) == Y_arr)[0][0]
    y_max = np.where(np.max(np.hstack(Y_valid)) == Y_arr)[0][0]
    
    
    # And now search the indices corresponding to the coordinates 
    x_matches = np.hstack([[np.where(i == X_arr[min(x_min-1, x_max+1):max(x_min-1, x_max+1)])[0][0] for i in row] for row in X_valid]).astype(int)
    y_matches = np.hstack([[np.where(i == Y_arr[min(y_min-1, y_max+1):max(y_min-1, y_max+1)])[0][0] for i in row] for row in Y_valid]).astype(int)
    
    # Create an array representing the glacier
    template = np.zeros((len(Y_arr[min(y_min-1, y_max+1):max(y_min-1, y_max+1)]), len( X_arr[min(x_min-1, x_max+1):max(x_min-1, x_max+1)])))
    template[y_matches, x_matches] = 1
except:
    pass

0it [00:00, ?it/s]


Plot the glacier shape to visualize which cells will be inverted/downloaded.

In [ ]:
try:
    plt.pcolormesh(X_arr[x_min-1:x_max+1], Y_arr[y_min-1:y_max+1], template)
except:
    pass

**Design Matrix**

The following function creates a unique design matrix for each datacube.
This design matrix will then be assembled in the inversion function.
The design matrix shares a common base for each point in the same datacube. But because each point has a different temporal completness, we modify the base design matrix to fit the point's data.

In [ ]:
def design_matrices(url, mission, lamb, derivative, day_interval, sdate, edate):

    # If you passed 'mission' as an argument, it grabs the appropriate values
    if mission:
        # Get the indices of the mission
        filt1 = np.where(data_dict[urls[url]]['zarr_store']['satellite_img1'][:] == mission)
        filt2 = np.where(data_dict[urls[url]]['zarr_store']['satellite_img2'][:] == mission)
        inds_mission = np.intersect1d(filt1[0],filt2[0])

        # Grab only the indices corresponding to the missions
        mid_dates = np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['mid_date'][:], dtype='timedelta64[D]')[inds_mission]
        im1 = np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['acquisition_date_img1'][:], dtype='timedelta64[D]')[inds_mission]
        im2 = np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['acquisition_date_img2'][:], dtype='timedelta64[D]')[inds_mission]
    else:
        # If 'None' was passed as a mission argument, we grab all the available data.
        inds_mission = None
        mid_dates = np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['mid_date'][:], dtype='timedelta64[D]')
        im1 = np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['acquisition_date_img1'][:], dtype='timedelta64[D]')
        im2 = np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['acquisition_date_img2'][:], dtype='timedelta64[D]')
    
    # Get some arrays
    index_sort = np.argsort(np.datetime64('1970-01-01') + np.array(data_dict[urls[url]]['zarr_store']['mid_date'][:], dtype='timedelta64[D]'))
    mid_dates = mid_dates[index_sort]
    im1 = im1[index_sort]
    im2 = im2[index_sort]

    # If sdate is later than the first available date, we find its corresponding index
    try:
        sdate_ind = np.where(mid_dates >= sdate)[0][0]
    except:
        sdate_ind = 0
    
    # If edate is sooner than the last available date, we find its corresponding index
    try:
        edate_ind = np.where(mid_dates > edate)[0][0]
    except:
        edate_ind = None
    
    # Create a False/True mask where True if the date is in the desired range
    mask_dates = np.full(mid_dates.shape, False)
    mask_dates[sdate_ind:edate_ind] = True

    # Keep only the values within the desired range
    mid_dates = mid_dates[mask_dates]
    im1 = im1[mask_dates]
    im2 = im2[mask_dates]

    # Check which im is the smallest (first image, it changes depending on ITS_LIVE's version)
    if im2[0] < im1[0]:
        temp = im1
        im1 = im2
        im2 = temp

    # Create the date array with the new interval dates
    dates_nonum = np.arange(mid_dates[0], mid_dates[-1], timedelta(days=day_interval)).astype(np.datetime64)

    # Convert to numerical
    dates = (dates_nonum - np.datetime64('1970-01-01T00:00:00Z'))/np.timedelta64(1, 's')
    dt_start = (im1 - np.datetime64('1970-01-01T00:00:00Z'))/np.timedelta64(1, 's')
    dt_end = (im2 - np.datetime64('1970-01-01T00:00:00Z'))/np.timedelta64(1, 's')

    # --------------- DESIGN MATRICES --------------- 

    # Initialize matrix
    A_m = np.zeros((mid_dates.shape[0],dates.shape[0]))

    # We have to iterate through the satellite pairs that actually gave a measurement
    for j in range(1, len(mid_dates)):
    # current contents of your for loop

        # Find the middate that is the closest to dt_start (supequal)
        start = np.argmin(np.abs(dates-dt_start[j]))

        # Find the middate that is closest to dt_end (infequal)
        end = np.argmin(dt_end[j] - dates[dates <= dt_end[j]])

        # Divide 1 by the amount of middates between d_start and d_end 
        if end == A_m.shape[1]-1: # If the mid_date is at the end of the array (acquisition im2 equals last mid_date)
            A_m[j, start:] = 1/(1+A_m.shape[1]-start)
        else: # If the measurement is in A's bounds temporally (we can have a satellite pair with the 2nd pair being outside of our mid_dates)
            A_m[j, start:end+1] = 1/(1+end-start) # Attribute to each pixel in the timescale of the satellite pair, the 1/amount of pixel in the pairmid_dates.shape


    # Initialize regularization matrix
    if derivative == 1:
        reg_mat_Inv = np.zeros((A_m.shape[1] -1, A_m.shape[1]))

        for j in range(A_m.shape[1] -1):
            reg_mat_Inv[j, j] = -lamb/day_interval
            reg_mat_Inv[j, j+1] = lamb/day_interval

    elif derivative == 2:
        # Initialize 2nd derivative regularization matrix
        reg_mat_Inv = np.zeros((A_m.shape[1] -1, A_m.shape[1]))

        for j in range(A_m.shape[1] -2):
            reg_mat_Inv[j, j] = lamb/(day_interval**2)
            reg_mat_Inv[j, j+1] = -2*lamb/(day_interval**2)
            reg_mat_Inv[j, j+2] = lamb/(day_interval**2)
            
    data_dict[urls[url]]['A_m'] = A_m
    data_dict[urls[url]]['reg_mat_Inv'] = reg_mat_Inv
    data_dict[urls[url]]['mission'] = mission
    data_dict[urls[url]]['index_sort'] = index_sort
    data_dict[urls[url]]['inds_mission'] = inds_mission
    data_dict[urls[url]]['dates'] = dates_nonum
    data_dict[urls[url]]['dates_noinv'] = mid_dates
    data_dict[urls[url]]['mask_dates']= mask_dates
            
    return data_dict

Create one design matrix per datacube

In [ ]:
try:

    for i in tqdm(range(len(urls))):
        design_matrices(i, mission, lamb, derivative, day_interval)
except:
    pass

0it [00:00, ?it/s]


**Point Inversion**

This function inputs a timeseries for a point and inverts it based on the design matrix build for the datacube the point belongs to.

In [ ]:
def Inv_reg(vObs, data, fillvalue):
    
    # Grab observed velocities
    vObs = vObs[data['index_sort']]
    vObs = vObs[data['mask_dates']]
    
    # Filter out the missions we don't want
    if mission:
        vObs = vObs[data['inds_mission']]  
    
    # Mask the NaNs so we don't compute the inversion for empty rows
    mask = np.logical_not(np.equal(vObs, fillvalue))
    
    # Create a masked velocity vector
    vObs_masked = vObs[mask]
    
    # Append regularization terms to dObs
    vObs_masked= np.hstack((vObs_masked, np.zeros((data['reg_mat_Inv'].shape[0]))))
     
    # Assemble the design matrix
    A_des = np.vstack((data['A_m'][mask], data['reg_mat_Inv']))
    
    # Invert the velocities
    vInv = np.linalg.solve(A_des.T@A_des,A_des.T@vObs_masked)
    #vInv = scipy.linalg.solve(A_des.T@A_des,A_des.T@vObs_masked, lower = False, check_finite=False, assume_a='gen')
    
    return vInv

Gather the dates at which we have an observation, spanning all the selected datacubes

In [ ]:
# Get the total amount of temporal steps
ind_tot = []
for i in urls:
    ind_tot.append(data_dict[i]['dates'])

ind_tot = np.unique(np.hstack(ind_tot))

for i in urls:
    data_dict[i]['ind_tot'] = np.array([np.where(c == ind_tot)[0][0] for c in data_dict[urls[0]]['dates']])

**Inversion Calculation**

The following cell loops through all the on-ice cells in the datacubes. It runs the inversion and outputs a timeseries linearized in time (based on your input at the beginning of the notebook).
It generates a .txt file that shows how many cells have been inverted. This is a check in case you get disconnected from your notebook but it still runs in the background. 
It also saves the output every 10000 iterations so if the script crashes, you don't have to start from scratch. 

Once the inversion is complete, it saves the output one last time as a netcdf file containing: vx, vy, time, x, y. 



In [ ]:
if GPU:

    import torch

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    '''
    # Migrate to torch & GPU
    for c in range(len(urls)):
        data_dict[urls[c]]['A_m'] = torch.from_numpy(data_dict[urls[c]]['A_m']).to(device)
        data_dict[urls[c]]['reg_mat_Inv'] = torch.from_numpy(data_dict[urls[c]]['reg_mat_Inv']).to(device)
    '''
    
    def Inv_reg_torch(vObs, data, fillvalue, device):
     
        # Grab observed velocities
        vObs = vObs[data['index_sort']]
        vObs = vObs[data['mask_dates']]
        
        # Filter out the missions we don't want
        if mission:
            vObs = vObs[data['inds_mission']]  
        
        # Mask the NaNs so we don't compute the inversion for empty rows
        mask = np.logical_not(np.equal(vObs, fillvalue))
        
        # Create a masked velocity vector
        vObs_masked = torch.from_numpy(vObs[mask])
        
        # Append regularization terms to dObs
        vObs_masked= torch.hstack((vObs_masked, torch.zeros((data['reg_mat_Inv'].shape[0])))).to(device)
         
        # Assemble the design matrix
        A_des = torch.vstack((data['A_m'][mask], data['reg_mat_Inv']))
        
        # Invert the velocities
        vInv = torch.linalg.solve(A_des.T@A_des,A_des.T@vObs_masked.double())
        
        return vInv
    
    vxInv = torch.zeros((len(ind_tot), template.shape[0], template.shape[1])).double().to(device)
    vyInv = torch.zeros((vxInv.shape)).double().to(device)
    
    # Define the total number of iterations
    total_iterations = len(y_matches)
    
    # Create a tqdm object with dynamic_ncols=False to suppress intermediate updates
    # Create a tqdm object with a larger mininterval to suppress intermediate updates
    progress_bar = tqdm(total=total_iterations, dynamic_ncols=False, mininterval=1.0)
    
    
    i = 0
    for c in range(len(urls)):
        valid_idx = data_dict[urls[c]]['valid_idx']
        fillvx = data_dict[urls[c]]['zarr_store']['vx'][:, valid_idx[0][0], valid_idx[1][0]].min() #data_dict[urls[c]]['zarr_store']['vx'].fill_value   fill_value is wrong in the new ITS_LIVE version
        fillvy = data_dict[urls[c]]['zarr_store']['vx'][:, valid_idx[0][0], valid_idx[1][0]].min() #data_dict[urls[c]]['zarr_store']['vy'].fill_value
    
        for V in range(len(valid_idx[0])):
            vxObs = data_dict[urls[c]]['zarr_store']['vx'][:, valid_idx[0][V], valid_idx[1][V]]
            vyObs = data_dict[urls[c]]['zarr_store']['vy'][:, valid_idx[0][V], valid_idx[1][V]]
            vxInv[data_dict[urls[c]]['ind_tot'], y_matches[i], x_matches[i]] = Inv_reg_torch(vxObs, data_dict[urls[c]], fillvx, device)
            vyInv[data_dict[urls[c]]['ind_tot'], y_matches[i], x_matches[i]] = Inv_reg_torch(vyObs, data_dict[urls[c]], fillvy, device)
            
            i += 1
            progress_bar.update(1)  # Update the progress bar

            if i%100 == 0:
                with open("Counter.txt", "w") as text_file:
                    text_file.write(f"Counter: {i}")
            
            if i%5000 == 0:
                
                print(f"Saved at {i}")
                # Get the names of all the glaciers in the datacube 
                #glac_names = np.hstack(np.array([glacier.id.values[0] for glacier in gdf_list]))
                #glac_names = '-'.join(glac_names)

                # Gather the projection of the cube
                #glac_proj = str(np.unique(np.hstack([data_dict[urls[i]]['proj_cube'] for i in range(len(urls))]))[0])

                # Create a new dataset with vx and vy, using attributes from 'ds'
                new_ds = xr.Dataset(
                    {
                        "vx": (["time", "y", "x"], vxInv.cpu().numpy()),
                        "vy": (["time", "y", "x"], vyInv.cpu().numpy()),
                    },
                    coords={
                        "time": ind_tot,
                        "x": X_arr[x_min-1:x_max+1],
                        "y": Y_arr[y_min-1:y_max+1],
                    },
                    attrs=data_dict[urls[0]]['zarr_store'].attrs,
                ).chunk({'time': 1, 'x': 100, 'y': 100})

                from dask.diagnostics import ProgressBar
                write_job = new_ds.to_netcdf(f'VelInv.nc', compute=False)
                with ProgressBar():
                    print(f"Writing to {'VelInv.nc'}")
                    write_job.compute()
    
    # Close the progress bar
    progress_bar.close()

    # Save the dataset
    new_ds = xr.Dataset(
        {
            "vx": (["time", "y", "x"], vxInv.cpu().numpy()),
            "vy": (["time", "y", "x"], vyInv.cpu().numpy()),
        },
        coords={
            "time": ind_tot,
            "x": X_arr[x_min-1:x_max+1],
            "y": Y_arr[y_min-1:y_max+1],
        },
        attrs=data_dict[urls[0]]['zarr_store'].attrs,
    ).chunk({'time': 1, 'x': 100, 'y': 100})
    
    from dask.diagnostics import ProgressBar
    write_job = new_ds.to_netcdf(f'VelInv.nc', compute=False)
    with ProgressBar():
        print(f"Writing to {'VelInv.nc'}")
        write_job.compute()



else:
    
    vxInv = np.zeros((len(ind_tot), template.shape[0], template.shape[1]))
    vyInv = np.zeros((vxInv.shape))
    
    # Define the total number of iterations
    total_iterations = len(y_matches)
    
    # Create a tqdm object with dynamic_ncols=False to suppress intermediate updates
    # Create a tqdm object with a larger mininterval to suppress intermediate updates
    progress_bar = tqdm(total=total_iterations, dynamic_ncols=False, mininterval=1.0)
    
    
    i = 0
    for c in range(len(urls)):
        valid_idx = data_dict[urls[c]]['valid_idx']
        fillvx = data_dict[urls[c]]['zarr_store']['vx'][:, valid_idx[0][0], valid_idx[1][0]].min() #data_dict[urls[c]]['zarr_store']['vx'].fill_value   fill_value is wrong in the new ITS_LIVE version
        fillvy = data_dict[urls[c]]['zarr_store']['vx'][:, valid_idx[0][0], valid_idx[1][0]].min() #data_dict[urls[c]]['zarr_store']['vy'].fill_value
        
        for V in range(len(valid_idx[0])):
            vxObs = data_dict[urls[c]]['zarr_store']['vx'][:, valid_idx[0][V], valid_idx[1][V]]
            vyObs = data_dict[urls[c]]['zarr_store']['vy'][:, valid_idx[0][V], valid_idx[1][V]]
            vxInv[data_dict[urls[c]]['ind_tot'], y_matches[i], x_matches[i]] = Inv_reg(vxObs, data_dict[urls[c]], fillvx)
            vyInv[data_dict[urls[c]]['ind_tot'], y_matches[i], x_matches[i]] = Inv_reg(vyObs, data_dict[urls[c]], fillvy)
            
            i += 1
            if i%100 == 0:
                with open("Counter.txt", "w") as text_file:
                    text_file.write(f"Counter: {i}")
            
            if i%10000 == 0:
                
                print(f"Saved at {i}")
                # Get the names of all the glaciers in the datacube 
                #glac_names = np.hstack(np.array([glacier.id.values[0] for glacier in gdf_list]))
                #glac_names = '-'.join(glac_names)

                # Gather the projection of the cube
                #glac_proj = str(np.unique(np.hstack([data_dict[urls[i]]['proj_cube'] for i in range(len(urls))]))[0])

                # Create a new dataset with vx and vy, using attributes from 'ds'
                new_ds = xr.Dataset(
                    {
                        "vx": (["time", "y", "x"], vxInv),
                        "vy": (["time", "y", "x"], vyInv),
                    },
                    coords={
                        "time": ind_tot,
                        "x": X_arr[x_min-1:x_max+1],
                        "y": Y_arr[y_min-1:y_max+1],
                    },
                    attrs=data_dict[urls[0]]['zarr_store'].attrs,
                ).chunk({'time': 1, 'x': 100, 'y': 100})

                from dask.diagnostics import ProgressBar
                write_job = new_ds.to_netcdf(f'Malaspina.nc', compute=False)
                with ProgressBar():
                    print(f"Writing to {'Malaspina.nc'}")
                    write_job.compute()
            progress_bar.update(1)  # Update the progress bar
    
    # Close the progress bar
    progress_bar.close()

    # Save the dataset
    new_ds = xr.Dataset(
        {
            "vx": (["time", "y", "x"], vxInv),
            "vy": (["time", "y", "x"], vyInv),
        },
        coords={
            "time": ind_tot,
            "x": X_arr[x_min-1:x_max+1],
            "y": Y_arr[y_min-1:y_max+1],
        },
        attrs=data_dict[urls[0]]['zarr_store'].attrs,
    ).chunk({'time': 1, 'x': 100, 'y': 100})

    from dask.diagnostics import ProgressBar
    write_job = new_ds.to_netcdf(f'Malaspina.nc', compute=False)
    with ProgressBar():
        print(f"Writing to {'Malaspina.nc'}")
        write_job.compute()